In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold

LOAD THE SAVED MODEL

In [2]:
class_num = 10

In [3]:
# Load the model from a GitHub repository
model = torch.hub.load(
    'OxWearables/ssl-wearables',  # Replace with the GitHub username and repository name
    'harnet5',          # Replace with the model function name in hubconf.py
    class_num=class_num,
    pretrained=True        # Set to True if you want to load pretrained weights
)

print(model)

Using cache found in C:\Users\jgm_6/.cache\torch\hub\OxWearables_ssl-wearables_main


131 Weights loaded
Resnet(
  (feature_extractor): Sequential(
    (layer1): Sequential(
      (0): Conv1d(3, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
      (1): ResBlock(
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (conv2): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (relu): ReLU(inplace=True)
      )
      (2): ResBlock(
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
      

In [4]:
# Freeze all layers in the model except for those in EvaClassifier
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the classifier layers
for param in model.classifier.linear1.parameters():
    param.requires_grad = True
# for param in model.classifier.linear_new1.parameters():
#     param.requires_grad = True
# for param in model.classifier.linear_new2.parameters():
#     param.requires_grad = True
for param in model.classifier.linear2.parameters():
    param.requires_grad = True

In [5]:
# Step 3: If necessary, verify the classifier's output size is now `num_classes`
print(model.classifier)  # Check the structure to confirm

EvaClassifier(
  (linear1): Linear(in_features=512, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=10, bias=True)
)


In [6]:
# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
# Load the state dictionary into the new instance
model.classifier.load_state_dict(torch.load(r'D:\Ali_Thesis\synthetic_data_generation\Data\process_data\acm_10_meta.pth'))

<All keys matched successfully>

In [8]:
# Step 3: If necessary, verify the classifier's output size is now `num_classes`
print(model)  # Check the structure to confirm

Resnet(
  (feature_extractor): Sequential(
    (layer1): Sequential(
      (0): Conv1d(3, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
      (1): ResBlock(
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (conv2): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (relu): ReLU(inplace=True)
      )
      (2): ResBlock(
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (conv2): Conv1d(6

LOAD 5 SECONDS SEQUENCES FROM H5PY DATASET

In [9]:
def read_h5(filepath, dataset_name):
    with h5py.File(filepath, 'r') as h5file:
        data = h5file[dataset_name][:]
    return data

In [10]:
X30 = read_h5(r'D:\Ali_Thesis\synthetic_data_generation\Data\Process_canada_data\P13_5_sec_30hz_acc_sequences.h5', 'data')
X30.shape

(417840, 3, 150)

In [11]:
# X_acc = X30[:, [1, 2, 3], :]  # Shape becomes (417840, 3, 320)
# X_acc
X_acc = X30
X_acc[0]

array([[-0.5020571 , -0.49900004, -0.50070229, -0.49946417, -0.50041967,
        -0.49967205, -0.5002488 , -0.499825  , -0.50010009, -0.49996656,
        -0.49995924, -0.50012314, -0.49978011, -0.50034175, -0.49948966,
        -0.50077759, -0.49869015, -0.50309197, -0.51390049, -0.49775895,
        -0.50122576, -0.49916226, -0.50062818, -0.49950613, -0.50039753,
        -0.49967973, -0.50027097, -0.49978087, -0.50016701, -0.49987827,
        -0.50006614, -0.5000171 , -0.49982383, -0.50061343, -0.49703134,
        -0.48071347, -0.49139242, -0.50159061, -0.49913016, -0.50075367,
        -0.49907797, -0.5016764 , -0.48731916, -0.4833831 , -0.48482223,
        -0.49909511, -0.50045683, -0.49976811, -0.4999715 , -0.50061386,
        -0.49623385, -0.4802237 , -0.49277483, -0.50133907, -0.49936095,
        -0.5005003 , -0.4994037 , -0.50114909, -0.48659834, -0.4833576 ,
        -0.50003231, -0.4997993 , -0.50021169, -0.49980058, -0.50018553,
        -0.49982653, -0.50017627, -0.49983128, -0.5

ONE SEQUENCE DID WORK !!

In [ ]:
# import numpy as np
# import torch

# all_logits = []

# # Example: A single sequence from your dataset (replace with your actual data)
# # Shape: (8, 320) for one sequence
# single_sequence = X_acc[0].astype(np.float32)

# # Convert to PyTorch tensor and add a batch dimension
# single_sequence_tensor = torch.tensor(single_sequence, dtype=torch.float32).unsqueeze(0)  # Shape: (1, 3, 320)

# # Move model and data to the same device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = classifier.to(device)
# single_sequence_tensor = single_sequence_tensor.to(device)

# # Perform inference
# model.eval()  # Set the model to evaluation mode
# with torch.no_grad():
#     output = model(single_sequence_tensor)  # Shape: (1, num_classes)


# all_logits.extend(output.cpu().tolist())  # Convert to list and store

# # Get predicted class
# predicted_class = torch.argmax(output, dim=-1).item()
# print(f"Predicted class for the sequence: {predicted_class}")


Predicted class for the sequence: 8


In [ ]:
# all_logits

[[-1.7843915224075317,
  -8.75704574584961,
  -9.047948837280273,
  0.5426174998283386,
  -7.941391468048096,
  -4.290830612182617,
  -5.96685266494751,
  -8.919855117797852,
  1.9559062719345093,
  -8.95846939086914]]

In [ ]:
# # Convert to tensor
# all_logits_tensor = torch.tensor(all_logits)

# # Find the highest class and logit for each sequence
# sequence_highest_classes = torch.argmax(all_logits_tensor, dim=-1).tolist()
# sequence_highest_logits = torch.max(all_logits_tensor, dim=-1).values.tolist()

In [ ]:
# # Output
# print(sequence_highest_classes)  # [3, 1, 1]
# print(sequence_highest_logits)   # [4.2, 3.4, 2.2]

[8]
[1.9559062719345093]


MORE SEQUENCES ...

In [12]:
sequences = X_acc.astype(np.float32)

In [13]:
sequences.shape[0]

417840

In [14]:
# Define the required parameters
all_logits = []
results = []

# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Process data in batches
with torch.no_grad():
    for i in range(sequences.shape[0]):
        sequence = sequences[i]
        
        # Convert batch to PyTorch tensor
        sequence_tensor = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0)
        
        # Move data and model to the same device
        sequence_tensor = sequence_tensor.to(device)
        
        # Classify the batch
        output = model(sequence_tensor)

        # Store logits for this batch
        all_logits.extend(output.cpu().tolist())  # Convert to list and store

        predicted_class = torch.argmax(output, dim=-1).item()

        # Append results
        results.append(predicted_class)

print("Classification completed using batches.")

logits_array = np.array(all_logits)
results_array = np.array(results)

print(f'The shape of activities logits: {logits_array.shape}')
print(f'The shape of activities: {results_array.shape}')

Classification completed using batches.
The shape of activities logits: (417840, 10)
The shape of activities: (417840,)


In [15]:
logits_array[0]

array([-26.08112526, -28.51968956, -26.82648087, -29.02114105,
       -25.72355461, -32.41719818, -30.02751541, -28.64265633,
       -25.18399811, -28.49422836])

In [16]:
results_array[0]

np.int64(8)

SAVE IN NPZ

In [17]:
np.savez_compressed('P13_classifications', array=results_array)

In [18]:
np.savez_compressed('P13_class_probs', array=logits_array)

- Best trial:
- Value: 0.7475
- Params:
  - lstm_layers: 2
  - hidden_size: 256
  - dropout: 0.26138219898271
  - activation: ReLU
  - lr: 0.0002829985470448175
  - weight_decay: 2.1242103361847734e-05
  - batch_size: 64
  - optimizer: Adam


In [ ]:
# # Define the model
# class CustomModel(nn.Module):
#     def __init__(self, feature_extractor, lstm_layers=1, hidden_size=256, dropout=0.2, activation='ReLU'):
#         super(CustomModel, self).__init__()
#         self.feature_extractor = feature_extractor

#         # Add a Linear layer to transform (batch_size, seq_length, 1) -> (batch_size, seq_length, 512)
#         self.feature_transformer = nn.Linear(1, 512)

#         self.lstm = nn.LSTM(input_size=512, hidden_size=hidden_size, num_layers=lstm_layers,
#                             batch_first=True, dropout=dropout if lstm_layers > 1 else 0)
        
#         activation_layer = {
#             'ReLU': nn.ReLU(),
#             'Tanh': nn.Tanh(),
#             'LeakyReLU': nn.LeakyReLU()
#         }[activation]
        
#         self.classifier = nn.Sequential(
#             nn.Linear(hidden_size, 512),
#             activation_layer,
#             nn.Linear(512, 1024),
#             activation_layer,
#             nn.Linear(1024, 512),
#             activation_layer,
#             nn.Linear(512, 10),
#             # activation_layer,
#             # nn.Linear(256, 206)
#         )
    
#     def forward(self, x):
#         features = self.feature_extractor(x)
#         # print("Feature extractor output shape:", features.shape)  # Debugging line to check the shape
#         features = self.feature_transformer(features)
        
#         lstm_out, _ = self.lstm(features)
#         lstm_last_output = lstm_out[:, -1, :]
#         output = self.classifier(lstm_last_output)
#         return output

In [ ]:
# # Create a new instance of the classifier
# classifier = CustomModel(
#     feature_extractor=feature_extractor,
#     lstm_layers=2,
#     hidden_size=256,
#     dropout=0.26138219898271,
#     activation='ReLU'
# )

# # Load the state dictionary into the new instance
# calssifier_path = r'D:\Rainbow_DQN_Test\Test_DQN\Data\process_data\best_model_10_meta_labels.pth'
# classifier.load_state_dict(torch.load(calssifier_path))
# # classifier.eval()  # Set the model to evaluation mode

# # Now `classifier` contains the saved weights
# print("Model loaded successfully!")

Model loaded successfully!


C:\Users\jgm_6\AppData\Local\Temp\ipykernel_18872\734222708.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state_dict(torch.load(calssifier_path))
